In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm
import numpy as np

In [2]:
data = pd.read_csv('non_nan.csv', )
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [3]:
def create_lags_cov(data, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)
    
    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs

In [4]:
def create_lags_returns(data, tickers, period):

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_tickers

In [5]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(data, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(data, tickers, period= period_returns)

# Creating dummy variables
data['Quater_dummy'] = 0
data['Quater_rippel'] = 0
data['Date'] = pd.to_datetime(data['Date'])
for index, row in data.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        data.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        data.loc[index, 'Quater_dummy'] = 1

In [6]:
data.columns

Index(['Date', 'unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth',
       'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE',
       'ROG', 'inflation_change', 'unemp_change', 'treasury_yield_change',
       'unemployment_rate_lag1', 'unemployment_rate_lag2',
       'unemployment_rate_lag3', 'CPI_lag1', 'CPI_lag2', 'CPI_lag3',
       'treasury_yield_lag1', 'treasury_yield_lag2', 'treasury_yield_lag3',
       'GDP_growth_lag1', 'GDP_growth_lag2', 'GDP_growth_lag3',
       'SP500_return_lag1', 'SP500_return_lag2', 'SP500_return_lag3',
       'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'AZN_lag1',
       'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1', 'JNJ_lag2', 'LLY_lag1',
       'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1', 'NVO_lag2', 'NVS_lag1',
       

In [7]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

In [8]:
def collect_prediction(X_train,y_train,X_test,y_test,estimator,alpha,model_name):
  estimator.fit(X_train,y_train)
  y_pred = estimator.predict(X_test)
  #print( "{model_name} alpha = {alpha:.2f},score = {score:.1f}".format(model_name=model_name, alpha=alpha , score= XGBQuantile.quantile_score(y_test, y_pred, alpha)) )

  return y_pred

In [9]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']
dates12m = ['2022-05-01']

In [10]:
to_exclude3 = ['Date']

for covariate in covs:
    to_exclude3.append(covariate)

for t in tickers:
    to_exclude3.append(t)

to_exclude3

['Date',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'AZN',
 'BMY',
 'JNJ',
 'LLY',
 'MRK',
 'NVO',
 'NVS',
 'PFE',
 'ROG']

In [11]:
def rolling_forecast_QR(data, dates, ticker, alpha, columns_to_exclude, horizon):
    t_loss = 0
    dta = data.copy()
    all_var_forecast = []
    all_returns = []

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude,axis=1)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude, axis=1)
        X_test = X_test.iloc[0:horizon]
        y_test = test_df[ticker][0:horizon]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)
        all_var_forecast.append(VaR_forecast)

        r = y_test
        all_returns.append(r)

        loss = tick_loss(alpha, r, VaR_forecast)
        t_loss += loss

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)
    all_returns = np.concatenate(all_returns)

    return t_loss, all_var_forecast, all_returns

In [12]:
from scipy.stats import norm
from arch import arch_model
import numpy as np

def rolling_forecast_GARCH(data, dates, ticker, alpha, columns_to_exclude, horizon):
    t_loss = 0
    dta = data.copy()
    all_var_forecast = []

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude, axis=1)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude, axis=1)
        X_test = X_test.iloc[horizon-1]
        y_test = test_df[ticker][horizon-1]

        garch_model = arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=horizon)
        cond_vol_forecast = np.sqrt(forecasts.variance.iloc[-1])
        mean_forecast = forecasts.mean.iloc[-1]

        VaR_forecast = mean_forecast + cond_vol_forecast * norm.ppf(alpha)
        all_var_forecast.append(VaR_forecast)

        r = y_test
        loss = tick_loss(alpha, r, VaR_forecast)
        t_loss += loss

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)

    return t_loss, all_var_forecast

In [13]:
import numpy as np
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

def rolling_forecast_XGB(data, ticker, alpha, exclude, dates, horizon): #,params, features=False, gcv=False)

    all_var_forecast = []

    t_loss = 0
    dta = data.copy()

    for d in dates:
      split_date = d
      train_df = dta[dta.Date <= split_date]
      train_df = train_df.dropna()
      test_df = dta[dta.Date > split_date]
      test_df = test_df.reset_index(drop=True)

      X_train = train_df.drop(columns=exclude, axis=1)
      X_test = test_df.drop(columns=exclude, axis=1)
      X_test = X_test.iloc[0:horizon]
      y_train = train_df[ticker]
      y_test = test_df[ticker][0:horizon]

      regressor = GradientBoostingRegressor()

      #y_pred = regressor.fit(X_train, y_train).predict(X_test)
      regressor.set_params(loss='quantile', alpha=0.05)
      y_lower = collect_prediction(X_train, y_train, X_test, y_test, estimator=regressor, alpha=0.05, model_name="Gradient Boosting")
      regressor.set_params(loss='quantile', alpha=0.95)
      #y_upper = collect_prediction(X_train, y_train, X_test, y_test, estimator=regressor, alpha=0.95, model_name="Gradient Boosting")
      tickloss = tick_loss(alpha, y_test, y_lower)
      #print(tickloss)
      t_loss += tickloss

      all_var_forecast.append(y_lower)

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)
    return t_loss, all_var_forecast

# 1 month

In [14]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
tick_losses_all = []
var_forecast_all =[]
returns_all_1m = []

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates1m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=1)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates1m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=1)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates1m, horizon=1)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all.append(var_forecast)

    # Store returns for each ticker
    returns_all_1m.append(all_returns)

tick_losses_all = np.array(tick_losses_all)
var_forecast_all = np.array(var_forecast_all)

print("Tick Losses Matrix:")
print(tick_losses_all)
print("VaR Forecasts All:")
print(var_forecast_all)
print("All Returns:")
print(returns_all_1m)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      7,   Func. Count:     41,   Neg. LLF: 892.7217703953827
Iteration:      8,   Func. Count:     46,   Neg. LLF: 892.7200619197179
Iteration:      9,   Func. Count:     51,   Neg. LLF: 892.7199879606185
Iteration:     10,   Func. Count:     56,   Neg. LLF: 892.7199864635309
Iteration:     11,   Func. Count:     60,   Neg. LLF: 892.7199864636577
Optimization terminated successfully    (Exit mode 0)
            Current function value: 892.7199864635309
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 11
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1244.0857254557427
Iteration:      2,   Func. Count:     13,   Neg. LLF: 900.9392194578554
Iteration:      3,   Func. Count:     19,   Neg. LLF: 898.9513385507807
Iteration:      4,   Func. Count:     26,   Neg. LLF: 895.8637164543424
Iteration:      5,   Func. Count:     31,   Neg. LLF: 895.8604740394894
Iteration:      6,   Func. Count:     36,   Neg. LLF: 895.845387216766

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1048.610023103777
            Iterations: 15
            Function evaluations: 87
            Gradient evaluations: 14
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1169.8733234051233
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1118.3661261766074
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1052.6559805610439
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1053.0010151331248
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1055.7506018207537
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1053.034347661995
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1052.4495124313562
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1052.176551037805
Iteration:      9,   Func. Count:     54,   Neg. LLF: 1052.1698786039997
Iteration:     10,   Func. Count:     59,   Neg. LLF: 1052.169349871926
Iteration:     11,   Func. Count:     64,   Neg. LLF: 1052.16924

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Tick Losses Matrix:
[[[0.89672138 0.75102131 0.76047544]]

 [[0.71467371 0.48696721 0.49885121]]

 [[2.31440119 0.46835748 0.60440059]]

 [[1.36682505 0.69379176 0.67465057]]

 [[0.65496638 0.58647176 0.6218841 ]]

 [[0.7442595  0.66693109 0.6294229 ]]

 [[1.49964732 0.45892824 0.51819829]]

 [[2.31069231 0.77356387 0.85779121]]

 [[4.4995135  3.98913715 4.03968005]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.06133914  -8.89037173  -7.34603066]
  [ -3.47685705  -8.66274921  -8.09896047]
  [ -9.30813624  -9.12655521  -8.11150573]
  [-12.5588184  -10.64292001 -10.81175824]
  [ -7.23167789 -10.35228813  -8.0707562 ]
  [ -7.97638183 -12.05033945  -7.6776094 ]
  [-15.34285013 -10.84558587  -8.46782904]
  [-14.46254905 -10.29198289  -7.70393457]
  [ -1.77012818  -9.69677902  -7.93625784]
  [ -7.88142787  -9.81479006  -7.92791712]
  [-19.04914778  -9.55508869 -10.37760251]]

 [[ -9.43470816  -9.73720522 -10.49375939]
  [-10.86444788  -9.61872766 -10.99602953]
  [ -8

In [15]:
tick_losses_all = np.array(tick_losses_all)
tick_losses_all_matrix = np.reshape(tick_losses_all, (9, 3))
tick_losses_all_matrix

array([[0.89672138, 0.75102131, 0.76047544],
       [0.71467371, 0.48696721, 0.49885121],
       [2.31440119, 0.46835748, 0.60440059],
       [1.36682505, 0.69379176, 0.67465057],
       [0.65496638, 0.58647176, 0.6218841 ],
       [0.7442595 , 0.66693109, 0.6294229 ],
       [1.49964732, 0.45892824, 0.51819829],
       [2.31069231, 0.77356387, 0.85779121],
       [4.4995135 , 3.98913715, 4.03968005]])

In [16]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker = []
for tick_losses in tick_losses_all_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker.append(weights)

# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.29645779 0.35397137 0.34957084]
Weights for Ticker 2: [0.25639446 0.37628484 0.3673207 ]
Weights for Ticker 3: [0.102346   0.50574554 0.39190846]
Weights for Ticker 4: [0.20015795 0.3943271  0.40551495]
Weights for Ticker 5: [0.31545862 0.35230135 0.33224003]
Weights for Ticker 6: [0.30317784 0.33833029 0.35849187]
Weights for Ticker 7: [0.13963203 0.45627787 0.4040901 ]
Weights for Ticker 8: [0.14968169 0.44711024 0.40320806]
Weights for Ticker 9: [0.30847339 0.34793995 0.34358666]


In [17]:
weights_array = np.array(weights_per_ticker)
weights_array

array([[0.29645779, 0.35397137, 0.34957084],
       [0.25639446, 0.37628484, 0.3673207 ],
       [0.102346  , 0.50574554, 0.39190846],
       [0.20015795, 0.3943271 , 0.40551495],
       [0.31545862, 0.35230135, 0.33224003],
       [0.30317784, 0.33833029, 0.35849187],
       [0.13963203, 0.45627787, 0.4040901 ],
       [0.14968169, 0.44711024, 0.40320806],
       [0.30847339, 0.34793995, 0.34358666]])

In [18]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts = []
row_sums_per_ticker = []
for ticker in range(len(var_forecast_all)):
    weights = weights_array[ticker]
    var_matrix = var_forecast_all[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts.append(weighted_var_matrix)
    row_sums = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker.append(row_sums)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums) in enumerate(zip(weighted_var_forecasts, row_sums_per_ticker)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.91633861 -3.23952835 -2.97807815]
 [-0.61110005 -3.14693704 -2.56795812]
 [-1.03074136 -3.06636519 -2.83116043]
 [-2.7594695  -3.23053923 -2.83554589]
 [-3.72315955 -3.76728895 -3.77947543]
 [-2.14388725 -3.66441359 -2.82130104]
 [-2.36466053 -4.26547514 -2.68386838]
 [-4.54850744 -3.83902687 -2.96010613]
 [-4.28753533 -3.64306726 -2.69307089]
 [-0.52476829 -3.43238213 -2.77428434]
 [-2.33651069 -3.47415466 -2.77136866]
 [-5.64726825 -3.38222781 -3.62770725]]
Row sums for Ticker 1:
[ -9.13394511  -6.32599521  -6.92826698  -8.82555462 -11.26992394
  -8.62960188  -9.31400405 -11.34764044 -10.62367349  -6.73143476
  -8.58203401 -12.65720332]
Weighted VaR Forecast Matrix for Ticker 2:
[[-2.41900687 -3.6639627  -3.85457508]
 [-2.78558421 -3.61938139 -4.03906931]
 [-2.18740664 -3.72433134 -4.03906915]
 [ 0.38428194 -4.2097491  -4.03392689]
 [-3.88343638 -3.93984401 -4.03128343]
 [-4.15801147 -4.11126789 -4.02911477]
 [-4.80492409 -3.84332939 -4

In [19]:
## Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for t, (weighted_var_matrix, row_sums) in enumerate(zip(weighted_var_forecasts, row_sums_per_ticker)):
    print(f"Ensemble forecasts for Ticker {t+1}:")
    print(row_sums)

Ensemble forecasts for Ticker 1:
[ -9.13394511  -6.32599521  -6.92826698  -8.82555462 -11.26992394
  -8.62960188  -9.31400405 -11.34764044 -10.62367349  -6.73143476
  -8.58203401 -12.65720332]
Ensemble forecasts for Ticker 2:
[ -9.93754466 -10.44403491  -9.95080714  -7.85939405 -11.85456382
 -12.29839413 -12.66950435 -11.60633447  -8.48232884 -11.40075546
 -12.7592852   -9.74611101]
Ensemble forecasts for Ticker 3:
[-5.57463635 -5.33046978 -4.29934995 -6.50681783 -6.43270763 -7.60503327
 -5.04001017 -6.80873503 -6.67186482 -7.42552108 -7.01271265 -7.25922517]
Ensemble forecasts for Ticker 4:
[ -9.22770487  -9.70864573  -7.35240146  -8.14599752  -8.71886405
  -9.34980828  -8.54384029  -8.45688271  -9.10979398  -7.84370786
  -9.39630223 -10.35955806]
Ensemble forecasts for Ticker 5:
[ -8.25965722 -13.70694974 -13.59051256  -6.93566317  -9.03480126
 -11.23930192 -13.79935562 -12.1139129  -10.58005487  -8.41269475
  -7.66617383  -9.69230169]
Ensemble forecasts for Ticker 6:
[ -8.67238643 -

In [20]:
for t in range(len(row_sums_per_ticker)):
    return_ticker = row_sums_per_ticker[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker)

Ensemble forecasts for Ticker 1: [ -9.13394511  -6.32599521  -6.92826698  -8.82555462 -11.26992394
  -8.62960188  -9.31400405 -11.34764044 -10.62367349  -6.73143476
  -8.58203401 -12.65720332]
Ensemble forecasts for Ticker 2: [ -9.93754466 -10.44403491  -9.95080714  -7.85939405 -11.85456382
 -12.29839413 -12.66950435 -11.60633447  -8.48232884 -11.40075546
 -12.7592852   -9.74611101]
Ensemble forecasts for Ticker 3: [-5.57463635 -5.33046978 -4.29934995 -6.50681783 -6.43270763 -7.60503327
 -5.04001017 -6.80873503 -6.67186482 -7.42552108 -7.01271265 -7.25922517]
Ensemble forecasts for Ticker 4: [ -9.22770487  -9.70864573  -7.35240146  -8.14599752  -8.71886405
  -9.34980828  -8.54384029  -8.45688271  -9.10979398  -7.84370786
  -9.39630223 -10.35955806]
Ensemble forecasts for Ticker 5: [ -8.25965722 -13.70694974 -13.59051256  -6.93566317  -9.03480126
 -11.23930192 -13.79935562 -12.1139129  -10.58005487  -8.41269475
  -7.66617383  -9.69230169]
Ensemble forecasts for Ticker 6: [ -8.67238643 -

In [21]:
return_ticker.flatten()

array([-10.64018944, -24.1409321 , -24.67399314, -16.74448631,
       -17.67339265, -16.57955692, -29.38713155, -10.81300841,
       -21.06089407, -23.55290258, -18.08752262, -13.37331183])

In [22]:
Var_list = []
for t in range(len(row_sums_per_ticker)):
    return_ticker = row_sums_per_ticker[t]
    Var_list.append(return_ticker)

In [23]:
ret_list = returns_all_1m

In [24]:
tick_loss_list=[]
for i in range(len(Var_list)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list[i], var=Var_list[i])
    tick_loss_list.append(tick_loss_ensemble)

In [25]:
tick_loss_list_1m=[]
for i in range(len(tick_loss_list)):
    tick_loss_list_1m.append(tick_loss_list[i]/12)

tick_loss_list_1m

[0.7398041788586817,
 0.4924148537249484,
 0.6001114178071268,
 0.7408887629774208,
 0.6198443723405682,
 0.6583464792358977,
 0.5599606903942407,
 0.9672869135070922,
 3.9265154745560156]

# 3 months

In [26]:
returns_all_3m = []
tick_losses_all_3m = []
var_forecast_all_3m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates3m, horizon=3)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_3m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_3m.append(var_forecast)

    # Store returns for each ticker
    returns_all_3m.append(all_returns)

tick_losses_all_3m = np.array(tick_losses_all_3m)
var_forecast_all_3m = np.array(var_forecast_all_3m)

print("Tick Losses Matrix:")
print(tick_losses_all_3m)
print("VaR Forecasts All:")
print(var_forecast_all_3m)
print("All Returns:")
print(returns_all_3m)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1239.3168332674327
Iteration:      2,   Func. Count:     13,   Neg. LLF: 897.2436887256465
Iteration:      3,   Func. Count:     19,   Neg. LLF: 896.728928670834
Iteration:      4,   Func. Count:     26,   Neg. LLF: 892.7751240041985
Iteration:      5,   Func. Count:     31,   Neg. LLF: 892.7716171571983
Iteration:      6,   Func. Count:     36,   Neg. LLF: 892.7534023585331
Iteration:      7,   Func. Count:     41,   Neg. LLF: 892.7217703953827
Iteration:      8,   Func. Count:     46,   Neg. LLF: 892.7200619197179
Iteration:      9,   Func. Count:     51,   Neg. LLF: 892.7199879606185
Iteration:     10,   Func. Count:     56,   Neg. LLF: 892.7199864635309
Iteration:     11,   Func. Count:     60,   Neg. LLF: 892.7199864636577
Optimization terminated successfully    (Exit mode 0)
            Current function value: 892.7199864635309
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 11

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warning

...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1161.1001574452678
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1087.735651160859
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1044.883827289639
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1046.7587090974273
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1050.487245194426
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1046.482878586167
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1045.429020908597
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1044.8013650643516
Iteration:      9,   Func. Count:     55,   Neg. LLF: 1044.4090513042167
Iteration:     10,   Func. Count:     61,   Neg. LLF: 1044.3920663986553
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1044.3919030758052
Iteration:     12,   Func. Count:     71,   Neg. LLF: 1044.3918989609401
Iteration:     13,   Func. Count:     76,   Neg. LLF: 1044.3918984136199
Optimization terminated successfully    (Exit mode 0)
  

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Tick Losses Matrix:
[[[ 2.95079751  2.42349257  2.50095217]]

 [[ 2.28026507  1.45078368  1.50524231]]

 [[ 6.78132128  1.38240144  1.593047  ]]

 [[ 4.46836782  2.05583184  1.9418676 ]]

 [[ 2.11000197  1.85296406  1.8566968 ]]

 [[ 2.24409617  1.95704452  1.98626   ]]

 [[ 4.74213431  1.44374109  1.55523838]]

 [[ 8.07806017  2.25544276  2.98129337]]

 [[14.08557797 13.54669229 13.88403966]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.75467099]
  [ -2.06133914  -8.89037173  -7.4787069 ]
  [ -3.47686179  -9.12229136  -8.19830572]
  [-10.29957562  -9.29385125  -7.4787069 ]
  [ -3.47685705  -8.66274921  -8.09896047]
  [-10.29963214  -8.94963189  -7.98915873]
  [-11.69836103  -9.16075273  -7.80844181]
  [ -9.30813624  -9.12655521  -8.26086029]
  [-12.65689204  -9.30123411  -7.94522473]
  [ -7.54087514  -9.43113514  -7.93711938]
  [-12.5588184  -10.64292001 -10.32682711]
  [ -7.44888531 -10.4701

In [27]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
tick_losses_all_3m = []
var_forecast_all_3m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates3m, horizon=3)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_3m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_3m.append(var_forecast)

    #returns of the tickers
    all_returns_3m = all_returns

    print(f'{t}-3m_return: {all_returns_3m} \n')

tick_losses_all_3m = np.array(tick_losses_all_3m)
var_forecast_all_3m = np.array(var_forecast_all_3m)


print("Tick Losses Matrix:")
print(tick_losses_all_3m)
print("VaR Forecasts All:")
print(var_forecast_all_3m)

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1239.3168332674327
Iteration:      2,   Func. Count:     13,   Neg. LLF: 897.2436887256465
Iteration:      3,   Func. Count:     19,   Neg. LLF: 896.728928670834
Iteration:      4,   Func. Count:     26,   Neg. LLF: 892.7751240041985
Iteration:      5,   Func. Count:     31,   Neg. LLF: 892.7716171571983
Iteration:      6,   Func. Count:     36,   Neg. LLF: 892.7534023585331
Iteration:      7,   Func. Count:     41,   Neg. LLF: 892.7217703953827
Iteration:      8,   Func. Count:     46,   Neg. LLF: 892.7200619197179
Iteration:      9,   Func. Count:     51,   Neg. LLF: 892.7199879606185
Iteration:     10,   Func. Count:     56,   Neg. LLF: 892.7199864635309
Iteration:     11,   Func. Count:     60,   Neg. LLF: 892.7199864636577
Optimization terminated successfully    (Exit mode 0)
            Current function value: 892.7199864635309
            Iterations: 11
            Function evaluations: 60
            Gradient evaluations: 11

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      6,   Func. Count:     37,   Neg. LLF: 1046.482878586167
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1045.429020908597
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1044.8013650643516
Iteration:      9,   Func. Count:     55,   Neg. LLF: 1044.4090513042167
Iteration:     10,   Func. Count:     61,   Neg. LLF: 1044.3920663986553
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1044.3919030758052
Iteration:     12,   Func. Count:     71,   Neg. LLF: 1044.3918989609401
Iteration:     13,   Func. Count:     76,   Neg. LLF: 1044.3918984136199
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1044.3918984136199
            Iterations: 13
            Function evaluations: 76
            Gradient evaluations: 13
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1165.6506883972468
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1108.8767328283766
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1049.122

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

ROG-3m_return: [ -1.2397349    2.73188743  -6.95635867   2.73188743  -6.95635867
  -3.44882614  -6.95635867  -3.44882614  -2.70795556  -3.44882614
  -2.70795556 -53.66506605  -2.70795556 -53.66506605   9.44607056
 -53.66506605   9.44607056  16.96832631   9.44607056  16.96832631
   5.4516805   16.96832631   5.4516805   11.02581254   5.4516805
  11.02581254  -1.51746671  11.02581254  -1.51746671  -2.16215955] 

Tick Losses Matrix:
[[[ 2.95079751  2.42349257  2.52912905]]

 [[ 2.28026507  1.45078368  1.47217645]]

 [[ 6.78132128  1.38240144  1.56704563]]

 [[ 4.46836782  2.05583184  1.94186844]]

 [[ 2.11000197  1.85296406  1.89214541]]

 [[ 2.24409617  1.95704452  1.98626   ]]

 [[ 4.74213431  1.44374109  1.55496461]]

 [[ 8.07806017  2.25544276  2.98143751]]

 [[14.08557797 13.54669229 13.84547985]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.51924071]
  [ -2.06133914  -8.89037173  -7.34612739

In [28]:
tick_losses_all_3m = np.array(tick_losses_all_3m)
tick_losses_all_3m_matrix = np.reshape(tick_losses_all_3m, (9, 3))
tick_losses_all_3m_matrix

array([[ 2.95079751,  2.42349257,  2.52912905],
       [ 2.28026507,  1.45078368,  1.47217645],
       [ 6.78132128,  1.38240144,  1.56704563],
       [ 4.46836782,  2.05583184,  1.94186844],
       [ 2.11000197,  1.85296406,  1.89214541],
       [ 2.24409617,  1.95704452,  1.98626   ],
       [ 4.74213431,  1.44374109,  1.55496461],
       [ 8.07806017,  2.25544276,  2.98143751],
       [14.08557797, 13.54669229, 13.84547985]])

In [29]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_3m = []
for tick_losses in tick_losses_all_3m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_3m.append(weights)

In [30]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_3m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.29548161 0.35977267 0.34474572]
Weights for Ticker 2: [0.24267989 0.38143142 0.37588869]
Weights for Ticker 3: [0.09772381 0.47938069 0.4228955 ]
Weights for Ticker 4: [0.18266259 0.39701869 0.42031872]
Weights for Ticker 5: [0.30732782 0.34995946 0.34271272]
Weights for Ticker 6: [0.30520503 0.34997131 0.34482366]
Weights for Ticker 7: [0.13634587 0.44784375 0.41581038]
Weights for Ticker 8: [0.13715471 0.49123126 0.37161403]
Weights for Ticker 9: [0.32710513 0.34011733 0.33277754]


In [31]:
weights_array_3m = np.array(weights_per_ticker_3m)

In [32]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_3m = []
row_sums_per_ticker_3m = []
for ticker in range(len(var_forecast_all_3m)):
    weights = weights_array_3m[ticker]
    var_matrix = var_forecast_all_3m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_3m.append(weighted_var_matrix)
    row_sums_3m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_3m.append(row_sums_3m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_3m) in enumerate(zip(weighted_var_forecasts_3m, row_sums_per_ticker_3m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_3m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.90673567 -3.29262161 -2.93697173]
 [-0.77954186 -3.35312157 -2.79834287]
 [-0.21487161 -3.39815455 -2.93697173]
 [-0.60908781 -3.1985128  -2.53254594]
 [-1.02734872 -3.28195115 -2.73330683]
 [-3.0433352  -3.34367371 -2.53254594]
 [-1.02734732 -3.11662044 -2.70369994]
 [-3.0433519  -3.21983299 -2.80533065]
 [-3.45665057 -3.2957885  -2.69806079]
 [-2.7503831  -3.28348517 -2.75422818]
 [-3.73987886 -3.34632986 -2.68406249]
 [-2.22818994 -3.3930647  -2.70366583]
 [-3.7108999  -3.82903179 -3.66883671]
 [-2.20100863 -3.76686884 -3.6206013 ]
 [-1.75744618 -3.71972474 -3.6206013 ]
 [-2.13682784 -3.72447038 -2.77163826]
 [-1.74586351 -3.68558146 -2.77163826]
 [-4.68763703 -3.65621182 -2.96302121]
 [-2.35687416 -4.33538284 -2.73548195]
 [-5.33626597 -4.15591277 -2.91286453]
 [-4.35754502 -4.01771767 -2.73548195]
 [-4.53353008 -3.90194542 -2.90037686]
 [-4.66803617 -3.82132915 -2.74070505]
 [-0.6363979  -3.76020404 -2.74070505]
 [-4.2734173  -3.7027

In [33]:
for t in range(len(row_sums_per_ticker_3m)):
    return_ticker_3m = row_sums_per_ticker_3m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_3m)

Ensemble forecasts for Ticker 1: [ -9.136329    -6.9310063   -6.54999789  -6.34014656  -7.0426067
  -8.91955485  -6.84766771  -9.06851554  -9.45049986  -8.78809644
  -9.77027121  -8.32492047 -11.2087684   -9.58847878  -9.09777223
  -8.63293648  -8.20308323 -11.30687007  -9.42773894 -12.40504327
 -11.11074464 -11.33585237 -11.23007037  -7.13730698 -10.69748005
  -6.67682469  -8.79972069  -6.75241231  -8.59552832 -12.31179483]
Ensemble forecasts for Ticker 2: [ -9.83335718 -10.40491886  -9.91421694 -10.05812895  -9.73690381
  -7.35348968  -9.59822123  -7.29279268 -11.46898372  -8.03162143
 -11.97869171 -11.55967252 -11.63530184 -11.43498963 -12.63721993
 -12.22619455 -13.01460342 -11.6132781  -12.56459066 -11.26735439
  -8.63212439 -11.75731941  -8.89402184 -11.20498595  -8.60317706
 -11.07422467 -12.75976558 -11.16811346 -12.76976694  -9.95286974]
Ensemble forecasts for Ticker 3: [-5.60613444 -5.70113861 -5.10378268 -5.33819671 -4.64257208 -5.90919112
 -4.36548984 -6.18996603 -6.0411006

In [34]:
return_ticker_3m.flatten()

array([-10.1255159 , -23.13173344, -26.95468585, -23.43064352,
       -25.93005964, -17.76182422, -25.24607133, -17.85451975,
       -16.69742351, -16.83498723, -16.84210902, -16.8715127 ,
       -17.70749144, -17.81200562, -11.25160794, -16.62222526,
       -12.82198237, -11.92755543, -28.86460844, -23.05708989,
       -33.19267197, -10.32645079, -20.85449112, -22.20312004,
       -21.27476965, -22.45919495, -18.54309361, -23.78739962,
       -18.61311778, -12.72273518])

In [35]:
Var_list_3m = []
for t in range(len(row_sums_per_ticker_3m)):
    return_ticker_3m = row_sums_per_ticker_3m[t]
    Var_list_3m.append(return_ticker_3m)

ret_list_3m = all_returns_3m

tick_loss_list_3=[]
for i in range(len(Var_list_3m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_3m[i], var=Var_list_3m[i])
    tick_loss_list_3.append(tick_loss_ensemble)

tick_loss_list_3m=[]
for i in range(len(tick_loss_list_3)):
    tick_loss_list_3m.append(tick_loss_list_3[i]/10)

tick_loss_list_3m

[1.1724799605318412,
 2.011987615726187,
 2.069705875135333,
 1.7234748851965018,
 0.5264463261933278,
 0.8908778458989233,
 0.5776957094043891,
 0.9858267882262075,
 2.502420151360751]

# 6 months

In [36]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
returns_all_6m = []
tick_losses_all_6m = []
var_forecast_all_6m =[]


for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=6)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates6m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=6)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates6m, horizon=6)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_6m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_6m.append(var_forecast)

    # Store returns for each ticker
    returns_all_6m.append(all_returns)

tick_losses_all_6m = np.array(tick_losses_all_6m)
var_forecast_all_6m = np.array(var_forecast_all_6m)


print("Tick Losses Matrix:")
print(tick_losses_all_3m)
print("VaR Forecasts All:")
print(var_forecast_all_3m)
print("All Returns:")
print(returns_all_3m)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1178.7314788173323
Iteration:      2,   Func. Count:     14,   Neg. LLF: 893.9662722206108
Iteration:      3,   Func. Count:     21,   Neg. LLF: 947.2792115942004
Iteration:      4,   Func. Count:     28,   Neg. LLF: 889.7749000463948
Iteration:      5,   Func. Count:     33,   Neg. LLF: 889.7675789803586
Iteration:      6,   Func. Count:     38,   Neg. LLF: 889.7537238994715
Iteration:      7,   Func. Count:     43,   Neg. LLF: 889.7361309667915
Iteration:      8,   Func. Count:     48,   Neg. LLF: 889.7274260931832
Iteration:      9,   Func. Count:     53,   Neg. LLF: 889.725416391847
Iteration:     10,   Func. Count:     58,   Neg. LLF: 889.7253145862583
Iteration:     11,   Func. Count:     62,   Neg. LLF: 889.7253145862426
Optimization terminated successfully    (Exit mode 0)
            Current function value: 889.7253145862583
            Iterations: 11
            Function evaluations: 62
            Gradient evaluations: 11

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
          

...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      3,   Func. Count:     20,   Neg. LLF: 1041.1938827489212
Iteration:      4,   Func. Count:     25,   Neg. LLF: 1050.0008570604436
Iteration:      5,   Func. Count:     31,   Neg. LLF: 1048.2288453655756
Iteration:      6,   Func. Count:     37,   Neg. LLF: 1043.837904983452
Iteration:      7,   Func. Count:     43,   Neg. LLF: 1042.206502283325
Iteration:      8,   Func. Count:     49,   Neg. LLF: 1041.4140764431854
Iteration:      9,   Func. Count:     55,   Neg. LLF: 1040.7792199491169
Iteration:     10,   Func. Count:     61,   Neg. LLF: 1040.6872092562228
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1040.6856185060324
Iteration:     12,   Func. Count:     71,   Neg. LLF: 1040.6854903803992
Iteration:     13,   Func. Count:     76,   Neg. LLF: 1040.6854265106572
Iteration:     14,   Func. Count:     81,   Neg. LLF: 22680.09054105734
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1040.6854256221252
            Iterat

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Tick Losses Matrix:
[[[ 2.95079751  2.42349257  2.52912905]]

 [[ 2.28026507  1.45078368  1.47217645]]

 [[ 6.78132128  1.38240144  1.56704563]]

 [[ 4.46836782  2.05583184  1.94186844]]

 [[ 2.11000197  1.85296406  1.89214541]]

 [[ 2.24409617  1.95704452  1.98626   ]]

 [[ 4.74213431  1.44374109  1.55496461]]

 [[ 8.07806017  2.25544276  2.98143751]]

 [[14.08557797 13.54669229 13.84547985]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.51924071]
  [ -2.06133914  -8.89037173  -7.34612739]
  [ -3.47686179  -9.12229136  -7.92847223]
  [-10.29957562  -9.29385125  -7.34612739]
  [ -3.47685705  -8.66274921  -7.84259188]
  [-10.29963214  -8.94963189  -8.1373909 ]
  [-11.69836103  -9.16075273  -7.82623445]
  [ -9.30813624  -9.12655521  -7.98915856]
  [-12.65689204  -9.30123411  -7.78562972]
  [ -7.54087514  -9.43113514  -7.84249292]
  [-12.5588184  -10.64292001 -10.6421532 ]
  [ -7.44888531 -10.4701

In [37]:
tick_losses_all_6m = np.array(tick_losses_all_6m)
tick_losses_all_6m_matrix = np.reshape(tick_losses_all_6m, (9, 3))
tick_losses_all_6m_matrix

array([[ 5.43825096,  4.01877127,  5.15427464],
       [ 4.45719481,  2.66627322,  3.12579508],
       [10.36730275,  2.47600197,  2.78987482],
       [ 7.09199951,  4.06037535,  3.76149694],
       [ 4.54496316,  3.36714969,  3.93376007],
       [ 4.72937822,  4.58264163,  4.1661157 ],
       [ 8.68946218,  3.3159731 ,  3.13997308],
       [13.82873898,  4.89922753,  6.57835587],
       [35.38864232, 35.95314858, 37.37650062]])

In [38]:

# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_6m = []
for tick_losses in tick_losses_all_6m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_6m.append(weights)

In [39]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_6m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.29340074 0.39703351 0.30956575]
Weights for Ticker 2: [0.24404326 0.40796582 0.34799093]
Weights for Ticker 3: [0.11231967 0.47029528 0.41738505]
Weights for Ticker 4: [0.21588686 0.37707585 0.40703729]
Weights for Ticker 5: [0.28529294 0.38508709 0.32961997]
Weights for Ticker 6: [0.31573421 0.32584404 0.35842175]
Weights for Ticker 7: [0.15654708 0.41022948 0.43322344]
Weights for Ticker 8: [0.16878224 0.47641093 0.35480682]
Weights for Ticker 9: [0.3411672  0.33581048 0.32302232]


In [40]:
weights_array_6m = np.array(weights_per_ticker_6m)

In [41]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_6m = []
row_sums_per_ticker_6m = []
for ticker in range(len(var_forecast_all_6m)):
    weights = weights_array_6m[ticker]
    var_matrix = var_forecast_all_6m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_6m.append(weighted_var_matrix)
    row_sums_6m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_6m.append(row_sums_6m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_6m) in enumerate(zip(weighted_var_forecasts_6m, row_sums_per_ticker_6m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_6m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.88626558 -3.63363095 -2.6372651 ]
 [-0.7740521  -3.70039675 -2.51278278]
 [-0.21335842 -3.7500937  -2.71014625]
 [-3.62079612 -3.78722471 -2.51278278]
 [-3.56069505 -3.81504255 -2.71014625]
 [-1.83743415 -3.83592455 -2.6372651 ]
 [-0.60479843 -3.52977551 -2.30942974]
 [-1.02011383 -3.62185538 -2.70225047]
 [-3.02190313 -3.6899704  -2.30942974]
 [-3.43231838 -3.74063187 -2.50679321]
 [-2.09963107 -3.77845751 -2.49008376]
 [-1.4421556  -3.80677815 -2.49008376]
 [-1.02011244 -3.43940174 -2.50590817]
 [-3.02191972 -3.55330378 -2.4110269 ]
 [-3.43230781 -3.63712583 -2.46042829]
 [-2.09962499 -3.6992437  -2.42651466]
 [-1.44216339 -3.74550265 -2.42651466]
 [-4.97491386 -3.78007185 -2.60843222]
 [-2.73101408 -3.62354827 -2.52532487]
 [-3.71354152 -3.69290165 -2.45958861]
 [-2.21249836 -3.74447671 -2.48702718]
 [-2.15924072 -3.78298213 -2.48702718]
 [-4.32442393 -3.81181164 -2.60759255]
 [-4.35436138 -3.83344147 -2.48702718]
 [-3.68476664 -4.2255

In [45]:
Var_list_6m = []
for t in range(len(row_sums_per_ticker_6m)):
    return_ticker_6m = row_sums_per_ticker_6m[t]
    Var_list_6m.append(return_ticker_6m)

ret_list_6m = returns_all_6m

tick_loss_list_6=[]
for i in range(len(Var_list_6m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_6m[i], var=Var_list_6m[i])
    tick_loss_list_6.append(tick_loss_ensemble)

tick_loss_list_6m=[]
for i in range(len(tick_loss_list_6)):
    tick_loss_list_6m.append(tick_loss_list_6[i]/7)

tick_loss_list_6m

[4.655722387640686,
 3.1023560608899303,
 3.070909391963849,
 3.798755351956821,
 4.077525742849188,
 4.314262356334645,
 3.5224969642535733,
 6.507424309307962,
 35.73262890327049]

# 9 months

In [46]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
returns_all_9m = []
tick_losses_all_9m = []
var_forecast_all_9m =[]


for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates9m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=9)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates9m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=9)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates9m, horizon=9)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_9m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_9m.append(var_forecast)

    # Store returns for each ticker
    returns_all_9m.append(all_returns)

tick_losses_all_9m = np.array(tick_losses_all_9m)
var_forecast_all_9m = np.array(var_forecast_all_9m)


print("Tick Losses Matrix:")
print(tick_losses_all_9m)
print("VaR Forecasts All:")
print(var_forecast_all_9m)
print("All Returns:")
print(returns_all_9m)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11
It

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
          

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      3,   Func. Count:     19,   Neg. LLF: 911.5634879907443
Iteration:      4,   Func. Count:     26,   Neg. LLF: 894.8689908106471
Iteration:      5,   Func. Count:     31,   Neg. LLF: 894.8683348870665
Iteration:      6,   Func. Count:     36,   Neg. LLF: 894.8682731527105
Iteration:      7,   Func. Count:     41,   Neg. LLF: 894.8678989972117
Iteration:      8,   Func. Count:     46,   Neg. LLF: 894.866039669116
Iteration:      9,   Func. Count:     51,   Neg. LLF: 894.8595683597949
Iteration:     10,   Func. Count:     56,   Neg. LLF: 894.8576466558918
Iteration:     11,   Func. Count:     61,   Neg. LLF: 894.8574935856548
Iteration:     12,   Func. Count:     66,   Neg. LLF: 894.8574870038208
Iteration:     13,   Func. Count:     71,   Neg. LLF: 894.8574862570197
Optimization terminated successfully    (Exit mode 0)
            Current function value: 894.8574862570197
            Iterations: 13
            Function evaluations: 71
            Gradient evaluations: 13


...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 867.9900312422881
Iteration:      2,   Func. Count:     13,   Neg. LLF: 859.2616055493777
Iteration:      3,   Func. Count:     20,   Neg. LLF: 843.7600474146132
Iteration:      4,   Func. Count:     27,   Neg. LLF: 833.3095140110556
Iteration:      5,   Func. Count:     32,   Neg. LLF: 833.230974403957
Iteration:      6,   Func. Count:     37,   Neg. LLF: 834.5977092957178
Iteration:      7,   Func. Count:     43,   Neg. LLF: 833.2088013618945
Iteration:      8,   Func. Count:     49,   Neg. LLF: 833.1985189269656
Iteration:      9,   Func. Count:     54,   Neg. LLF: 833.1985119125833
Iteration:     10,   Func. Count:     58,   Neg. LLF: 833.1985119122887
Optimization terminated successfully    (Exit mode 0)
            Current function value: 833.1985119125833
            Iterations: 10
            Function evaluations: 58
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 922.0800563778539


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1215.021

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False.

Tick Losses Matrix:
[[[ 8.18461796  5.72395866  7.734875  ]]

 [[ 6.93617375  3.64972198  4.62522793]]

 [[20.60742827  2.50038446  5.54193772]]

 [[13.67942271  7.56808716  5.80644189]]

 [[ 6.42922845  4.68162065  5.80591236]]

 [[ 6.89263536  5.91161015  5.94513772]]

 [[14.61166592  4.7635035   4.61514023]]

 [[16.81566142  3.80591614  9.18644608]]

 [[45.10503871  8.94014037 43.82317246]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.51924071]
  [-12.34078718  -9.53880365  -8.11712154]
  [-12.13594422  -9.60886785  -8.51924071]
  [ -6.26254089  -9.6614629   -8.51924071]
  [ -2.7415266   -9.70100222  -8.51924071]
  [-20.55341384  -9.73075894  -8.51924071]
  [-14.06886188  -9.75317152  -8.51924071]
  [ -2.06133914  -8.89037173  -7.42751592]
  [ -3.47686179  -9.12229136  -8.65688762]
  [-10.29957562  -9.29385125  -7.53569854]
  [-11.69839703  -9.42145122  -7.82052119]
  [ -7.15618868  -9.5167

In [47]:
tick_losses_all_9m = np.array(tick_losses_all_9m)
tick_losses_all_9m_matrix = np.reshape(tick_losses_all_9m, (9, 3))
tick_losses_all_9m_matrix

array([[ 8.18461796,  5.72395866,  7.734875  ],
       [ 6.93617375,  3.64972198,  4.62522793],
       [20.60742827,  2.50038446,  5.54193772],
       [13.67942271,  7.56808716,  5.80644189],
       [ 6.42922845,  4.68162065,  5.80591236],
       [ 6.89263536,  5.91161015,  5.94513772],
       [14.61166592,  4.7635035 ,  4.61514023],
       [16.81566142,  3.80591614,  9.18644608],
       [45.10503871,  8.94014037, 43.82317246]])

In [48]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_9m = []
for tick_losses in tick_losses_all_9m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_9m.append(weights)

In [49]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_9m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.28669458 0.40994104 0.30336438]
Weights for Ticker 2: [0.22726727 0.43191379 0.34081894]
Weights for Ticker 3: [0.07715957 0.6359263  0.28691413]
Weights for Ticker 4: [0.19367015 0.35006148 0.45626837]
Weights for Ticker 5: [0.28730265 0.39455021 0.31814714]
Weights for Ticker 6: [0.30072272 0.35062732 0.34864996]
Weights for Ticker 7: [0.13824665 0.42406052 0.43769283]
Weights for Ticker 8: [0.13795422 0.6095225  0.25252328]
Weights for Ticker 9: [0.14135319 0.71315892 0.14548789]


In [50]:
weights_array_9m = np.array(weights_per_ticker_9m)

In [51]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_9m = []
row_sums_per_ticker_9m = []
for ticker in range(len(var_forecast_all_9m)):
    weights = weights_array_9m[ticker]
    var_matrix = var_forecast_all_9m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_9m.append(weighted_var_matrix)
    row_sums_9m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_9m.append(row_sums_9m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_9m) in enumerate(zip(weighted_var_forecasts_9m, row_sums_per_ticker_9m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_9m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.82029513 -3.75176002 -2.58443419]
 [-0.75635984 -3.82069637 -2.46244556]
 [-0.20848176 -3.87200896 -2.58443419]
 [-3.53803675 -3.91034711 -2.46244556]
 [-3.47930938 -3.9390693  -2.58443419]
 [-1.7954365  -3.96063017 -2.58443419]
 [-0.78598081 -3.97683896 -2.58443419]
 [-5.89255226 -3.98903746 -2.58443419]
 [-4.03346639 -3.99822529 -2.58443419]
 [-0.59097475 -3.64452825 -2.25324378]
 [-0.99679742 -3.73960162 -2.62619137]
 [-2.95283246 -3.80993106 -2.28606253]
 [-3.35386698 -3.86223953 -2.37246758]
 [-2.05164048 -3.90129488 -2.40274267]
 [-1.40919271 -3.93053622 -2.40274267]
 [-4.86121131 -3.95247419 -2.51343942]
 [-3.95008094 -3.96895753 -2.62619137]
 [-0.38300441 -3.98135621 -2.28351887]
 [-0.99679606 -3.55121644 -2.52344243]
 [-2.95284867 -3.66882142 -2.4236262 ]
 [-3.35385665 -3.75536852 -2.37713017]
 [-2.05163453 -3.81950584 -2.36865396]
 [-1.40920032 -3.86726866 -2.36865396]
 [-4.86120385 -3.9029617  -2.52216922]
 [-3.95008805 -3.9297

In [52]:
for t in range(len(row_sums_per_ticker_9m)):
    return_ticker_9m = row_sums_per_ticker_9m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_9m)

Ensemble forecasts for Ticker 1: [ -9.15648934  -7.03950177  -6.66492491  -9.91082941 -10.00281287
  -8.34050087  -7.34725396 -12.46602391 -10.61612588  -6.48874678
  -7.36259041  -9.04882606  -9.58857408  -8.35567802  -7.7424716
 -11.32712492 -10.54522983  -6.64787949  -7.07145493  -9.0452963
  -9.48635534  -8.23979433  -7.64512294 -11.28633477 -10.403233
  -6.70142363  -8.50223836  -8.88470903  -9.81476101  -8.39679127
  -8.38450815 -10.67949548 -10.58155542  -6.62708209  -8.55618658
 -11.76245652]
Ensemble forecasts for Ticker 2: [ -9.86433487 -10.44363279 -10.00334642  -7.87149172 -11.59144511
 -11.28571512 -12.51240285 -11.29959478  -8.60711243 -10.37124965
 -10.11840277  -7.90723461 -11.79546634 -11.54486129 -12.66770488
 -11.42495671  -8.86905165 -10.95545696  -9.63730156  -7.51444041
 -11.44140446 -11.20827787 -12.33910333 -11.1000181   -8.54579984
 -10.63298129 -12.31316429  -8.23436462 -11.91628917 -11.51734243
 -12.69584098 -11.42562491  -8.88110697 -11.02348718 -12.65780302

In [ ]:
return_ticker_9m.flatten()

array([-14.01313465, -19.34421294, -21.27497166, -16.89562428,
       -17.00480947, -17.33741372, -15.71918808, -14.36280538,
       -18.16045827, -19.50258636, -20.60444257, -17.12714838,
       -16.94821997, -16.95850178, -15.97271776, -15.07599291,
       -17.93215705, -19.19532556, -20.03025393, -16.92370637,
       -16.47172042, -16.55645343, -14.90232389, -14.62373751,
       -17.63680718, -18.81680808, -16.20545523, -16.38233753,
       -16.46309257, -16.53982386, -14.81237418, -14.54278514,
       -17.45752024, -18.6691209 , -16.16708155, -13.77742707])

In [53]:
Var_list_9m = []
for t in range(len(row_sums_per_ticker_9m)):
    return_ticker_9m = row_sums_per_ticker_9m[t]
    Var_list_9m.append(return_ticker_9m)

ret_list_9m = returns_all_9m

tick_loss_list_9=[]
for i in range(len(Var_list_9m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_9m[i], var=Var_list_9m[i])
    tick_loss_list_9.append(tick_loss_ensemble)

tick_loss_list_9m=[]
for i in range(len(tick_loss_list_9)):
    tick_loss_list_9m.append(tick_loss_list_9[i]/4)

tick_loss_list_9m

[6.8305677957678785,
 4.539512473255421,
 4.909808073274735,
 6.3335515150240385,
 5.8540316544838555,
 6.128528547790986,
 5.222771458086991,
 7.472487328826629,
 43.2372360139027]

# 12 months

In [54]:
# Computing tick losses and var forecast at horizon=1 for all tickers and all models
returns_all_12m = []
tick_losses_all_12m = []
var_forecast_all_12m =[]

for t in tickers:
    # Get tick loss and forecasts for each model
    t_loss_qr, all_var_forecast_qr, all_returns = rolling_forecast_QR(data, dates12m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=12)
    t_loss_garch, all_var_forecast_garch = rolling_forecast_GARCH(data, dates12m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=12)
    t_loss_xgboost, all_var_forecast_xgboost = rolling_forecast_XGB(data, t, alpha=0.05, exclude=to_exclude3, dates=dates12m, horizon=12)

    # Combine tick losses into a matrix
    tick_losses = np.column_stack((t_loss_qr, t_loss_garch, t_loss_xgboost))
    tick_losses_all_12m.append(tick_losses)

    # Combine var forecasts into a matrix
    var_forecast = np.column_stack((all_var_forecast_qr,all_var_forecast_garch,all_var_forecast_xgboost))
    var_forecast_all_12m.append(var_forecast)

    # Store returns for each ticker
    returns_all_12m.append(all_returns)

tick_losses_all_12m = np.array(tick_losses_all_12m)
var_forecast_all_12m = np.array(var_forecast_all_12m)


print("Tick Losses Matrix:")
print(tick_losses_all_12m)
print("VaR Forecasts All:")
print(var_forecast_all_12m)
print("All Returns:")
print(returns_all_12m)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 874.1785677557702
Iteration:      2,   Func. Count:     12,   Neg. LLF: 1200.4721531726964
Iteration:      3,   Func. Count:     19,   Neg. LLF: 870.6852289593519
Iteration:      4,   Func. Count:     26,   Neg. LLF: 867.8399619926613
Iteration:      5,   Func. Count:     32,   Neg. LLF: 867.828481407204
Iteration:      6,   Func. Count:     37,   Neg. LLF: 867.8058039900275
Iteration:      7,   Func. Count:     42,   Neg. LLF: 867.748173849943
Iteration:      8,   Func. Count:     47,   Neg. LLF: 867.7462745676646
Iteration:      9,   Func. Count:     52,   Neg. LLF: 867.7461706287772
Iteration:     10,   Func. Count:     57,   Neg. LLF: 867.746169625292
Iteration:     11,   Func. Count:     61,   Neg. LLF: 867.7461696251956
Optimization terminated successfully    (Exit mode 0)
            Current function value: 867.746169625292
            Iterations: 11
            Function evaluations: 61
            Gradient evaluations: 11


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 904.2346809763922
Iteration:      2,   Func. Count:     13,   Neg. LLF: 909.8940809007032
Iteration:      3,   Func. Count:     19,   Neg. LLF: 901.9676011216195
Iteration:      4,   Func. Count:     26,   Neg. LLF: 885.2200651421106
Iteration:      5,   Func. Count:     31,   Neg. LLF: 885.2195442810739
Iteration:      6,   Func. Count:     36,   Neg. LLF: 885.2195104425602
Iteration:      7,   Func. Count:     41,   Neg. LLF: 885.2193138255832
Iteration:      8,   Func. Count:     46,   Neg. LLF: 885.2183522300605
Iteration:      9,   Func. Count:     51,   Neg. LLF: 885.2147616595502
Iteration:     10,   Func. Count:     56,   Neg. LLF: 885.2128950597746
Iteration:     11,   Func. Count:     61,   Neg. LLF: 885.2127609855501
Iteration:     12,   Func. Count:     66,   Neg. LLF: 885.2127585731427
Iteration:     13,   Func. Count:     70,   Neg. LLF: 885.2127585730582
Optimization terminated successfully    (Exit mode 0)
          

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 3737.673074947868
Iteration:      2,   Func. Count:     14,   Neg. LLF: 1059.3458407887865
Iteration:      3,   Func. Count:     20,   Neg. LLF: 990.3442313922687
Iteration:      4,   Func. Count:     26,   Neg. LLF: 774.7400448569347
Iteration:      5,   Func. Count:     32,   Neg. LLF: 779.060699049972
Iteration:      6,   Func. Count:     38,   Neg. LLF: 771.3832501664328
Iteration:      7,   Func. Count:     44,   Neg. LLF: 769.7415752103974
Iteration:      8,   Func. Count:     50,   Neg. LLF: 765.6335026311029
Iteration:      9,   Func. Count:     55,   Neg. LLF: 765.6297294381544
Iteration:     10,   Func. Count:     60,   Neg. LLF: 765.6289482309151
Iteration:     11,   Func. Count:     65,   Neg. LLF: 765.628625130922
Iteration:     12,   Func. Count:     70,   Neg. LLF: 765.6283320636917
Iteration:     13,   Func. Count:     75,   Neg. LLF: 765.6283000831261
Iteration:     14,   Func. Count:     80,   Neg. LLF: 765.6282984

...

Iteration:      1,   Func. Count:      6,   Neg. LLF: 867.9900312422881
Iteration:      2,   Func. Count:     13,   Neg. LLF: 859.2616055493777
Iteration:      3,   Func. Count:     20,   Neg. LLF: 843.7600474146132
Iteration:      4,   Func. Count:     27,   Neg. LLF: 833.3095140110556
Iteration:      5,   Func. Count:     32,   Neg. LLF: 833.230974403957
Iteration:      6,   Func. Count:     37,   Neg. LLF: 834.5977092957178
Iteration:      7,   Func. Count:     43,   Neg. LLF: 833.2088013618945
Iteration:      8,   Func. Count:     49,   Neg. LLF: 833.1985189269656
Iteration:      9,   Func. Count:     54,   Neg. LLF: 833.1985119125833
Iteration:     10,   Func. Count:     58,   Neg. LLF: 833.1985119122887
Optimization terminated successfully    (Exit mode 0)
            Current function value: 833.1985119125833
            Iterations: 10
            Function evaluations: 58
            Gradient evaluations: 10


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1210.4827289658347
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1011.3744844764094
Iteration:      3,   Func. Count:     19,   Neg. LLF: 1009.5684780724663
Iteration:      4,   Func. Count:     24,   Neg. LLF: 1009.5683778425991
Iteration:      5,   Func. Count:     29,   Neg. LLF: 1009.5682735267333
Iteration:      6,   Func. Count:     34,   Neg. LLF: 1009.5680105828636
Iteration:      7,   Func. Count:     39,   Neg. LLF: 1009.5663903221539
Iteration:      8,   Func. Count:     44,   Neg. LLF: 1009.5628278473197
Iteration:      9,   Func. Count:     49,   Neg. LLF: 1009.562807759047
Iteration:     10,   Func. Count:     53,   Neg. LLF: 1009.5628077585338
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1009.562807759047
            Iterations: 10
            Function evaluations: 53
            Gradient evaluations: 10


/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Tick Losses Matrix:
[[[11.20032919  5.64811163  9.18947497]]

 [[ 8.44300384  4.76410303  5.83211426]]

 [[27.74149871  0.78862259  6.33150714]]

 [[19.24636627 11.73234121  8.00649684]]

 [[ 7.84057241  3.18186983  7.31731745]]

 [[ 9.00189077  3.12873027  7.42138368]]

 [[21.74261694  0.61556187  6.13896092]]

 [[18.61012257  6.18731604 10.37365793]]

 [[54.87740592  8.93592537 46.41851326]]]
VaR Forecasts All:
[[[ -9.83728109  -9.15195024  -8.51924071]
  [ -2.6382077   -9.32011188  -8.11712154]
  [ -0.72719114  -9.44528254  -8.75467099]
  [-12.34078718  -9.53880365  -8.11712154]
  [-12.13594422  -9.60886785  -8.75467099]
  [ -6.26254089  -9.6614629   -8.51924071]
  [ -2.7415266   -9.70100222  -8.51924071]
  [-20.55341384  -9.73075894  -8.75467099]
  [-14.06886188  -9.75317152  -8.75467099]
  [  1.35556108  -9.77006275  -8.11712154]
  [ -7.29180996  -9.78279858  -8.75467099]
  [-20.67749581  -9.79240455  -8.75467099]]

 [[ -9.43470816  -9.73720522 -10.00648067]
  [-10.8634487  -10.32

In [55]:
tick_losses_all_12m = np.array(tick_losses_all_12m)
tick_losses_all_12m_matrix = np.reshape(tick_losses_all_12m, (9, 3))
tick_losses_all_12m_matrix

array([[11.20032919,  5.64811163,  9.18947497],
       [ 8.44300384,  4.76410303,  5.83211426],
       [27.74149871,  0.78862259,  6.33150714],
       [19.24636627, 11.73234121,  8.00649684],
       [ 7.84057241,  3.18186983,  7.31731745],
       [ 9.00189077,  3.12873027,  7.42138368],
       [21.74261694,  0.61556187,  6.13896092],
       [18.61012257,  6.18731604, 10.37365793],
       [54.87740592,  8.93592537, 46.41851326]])

In [56]:
# Compute weights for each particular ticker with higher weight to lower tick loss models
weights_per_ticker_12m = []
for tick_losses in tick_losses_all_12m_matrix:
    inverted_tick_losses = 1 / tick_losses
    total_inverted_loss = np.sum(inverted_tick_losses)
    weights = inverted_tick_losses / total_inverted_loss
    weights_per_ticker_12m.append(weights)

In [57]:
# Print the weights for each particular ticker
for ticker, weights in enumerate(weights_per_ticker_12m):
    print(f"Weights for Ticker {ticker+1}: {weights}")

Weights for Ticker 1: [0.23799078 0.47194094 0.29006827]
Weights for Ticker 2: [0.23697315 0.41996683 0.34306002]
Weights for Ticker 3: [0.02465565 0.86731562 0.10802873]
Weights for Ticker 4: [0.19824366 0.3252096  0.47654674]
Weights for Ticker 5: [0.22047561 0.54328276 0.23624163]
Weights for Ticker 6: [0.19645836 0.56524421 0.23829743]
Weights for Ticker 7: [0.02508572 0.88606714 0.08884714]
Weights for Ticker 8: [0.17236135 0.51842605 0.30921261]
Weights for Ticker 9: [0.1201426  0.73782109 0.14203631]


In [58]:
weights_array_12m = np.array(weights_per_ticker_12m)

In [59]:
# Multiply each VaR forecast matrix by the corresponding weights
weighted_var_forecasts_12m = []
row_sums_per_ticker_12m = []
for ticker in range(len(var_forecast_all_12m)):
    weights = weights_array_12m[ticker]
    var_matrix = var_forecast_all_12m[ticker]
    weighted_var_matrix = var_matrix * np.transpose(weights)
    weighted_var_forecasts_12m.append(weighted_var_matrix)
    row_sums_12m = np.sum(weighted_var_matrix, axis=1)
    row_sums_per_ticker_12m.append(row_sums_12m)

# Print the weighted VaR forecast matrices and their row sums (weithed ensemble predictions)
for ticker, (weighted_var_matrix, row_sums_12m) in enumerate(zip(weighted_var_forecasts_12m, row_sums_per_ticker_12m)):
    print(f"Weighted VaR Forecast Matrix for Ticker {ticker+1}:")
    print(weighted_var_matrix)
    print(f"Row sums for Ticker {ticker+1}:")
    print(row_sums_12m)

Weighted VaR Forecast Matrix for Ticker 1:
[[-2.34118224 -4.31918003 -2.47116143]
 [-0.62786912 -4.39854239 -2.35451942]
 [-0.17306479 -4.45761555 -2.53945229]
 [-2.93699362 -4.50175199 -2.35451942]
 [-2.88824289 -4.53481815 -2.53945229]
 [-1.49042702 -4.55963991 -2.47116143]
 [-0.65245807 -4.57830014 -2.47116143]
 [-4.89152308 -4.59234355 -2.53945229]
 [-3.34825947 -4.60292096 -2.53945229]
 [ 0.32261104 -4.61089263 -2.35451942]
 [-1.73538357 -4.61690319 -2.53945229]
 [-4.92105345 -4.62143664 -2.53945229]]
Row sums for Ticker 1:
[ -9.13152371  -7.38093093  -7.17013263  -9.79326503  -9.96251333
  -8.52122837  -7.70191964 -12.02331892 -10.49063273  -6.64280101
  -8.89173905 -12.08194238]
Weighted VaR Forecast Matrix for Ticker 2:
[[-2.23577253 -4.08930319 -3.43282346]
 [-2.57434568 -4.33685299 -3.43282346]
 [-1.99920446 -4.4450728  -3.43282346]
 [ 0.27554824 -4.49342655 -3.43282346]
 [-3.57989232 -4.51522924 -3.43282346]
 [-3.25052114 -4.52509937 -3.43282346]
 [-4.52479677 -4.52957556 -3

In [60]:
for t in range(len(row_sums_per_ticker_12m)):
    return_ticker_12m = row_sums_per_ticker_12m[t]
    print(f"Ensemble forecasts for Ticker {t+1}:",return_ticker_12m)

Ensemble forecasts for Ticker 1: [ -9.13152371  -7.38093093  -7.17013263  -9.79326503  -9.96251333
  -8.52122837  -7.70191964 -12.02331892 -10.49063273  -6.64280101
  -8.89173905 -12.08194238]
Ensemble forecasts for Ticker 2: [ -9.75789918 -10.34402214  -9.87710071  -7.65070177 -11.52794502
 -11.20844397 -12.4871958  -11.22244541  -8.4149087  -10.74947912
 -12.37254805  -9.30316454]
Ensemble forecasts for Ticker 3: [-6.3736059  -6.38255421 -6.21758622 -6.54997394 -6.54470607 -6.56256692
 -6.45918933 -6.74834493 -6.42278099 -6.55475801 -6.62701676 -6.68986843]
Ensemble forecasts for Ticker 4: [ -9.14105259 -10.02405909  -7.64725692  -9.0863038   -8.25792099
  -9.88926078  -8.77645103  -9.78542814  -8.87086697  -7.53034742
  -8.98880473 -11.02545008]
Ensemble forecasts for Ticker 5: [ -8.70340101 -12.58843316 -12.56326982  -7.96480831  -9.32547892
 -10.88937391 -12.94016776  -8.98596736 -10.28793946  -8.93074165
  -8.36307338  -9.90672255]
Ensemble forecasts for Ticker 6: [ -8.76326009 -

In [61]:
return_ticker_12m.flatten()

array([-14.42712984, -19.44194   , -20.5494814 , -16.99832578,
       -16.96159545, -17.24192698, -16.28969539, -14.75777641,
       -17.86237037, -18.86690839, -16.6495641 , -14.71306321])

In [62]:
Var_list_12m = []
for t in range(len(row_sums_per_ticker_12m)):
    return_ticker_12m = row_sums_per_ticker_12m[t]
    Var_list_12m.append(return_ticker_12m)

ret_list_12m = returns_all_12m

tick_loss_list_12=[]
for i in range(len(Var_list_12m)):
    tick_loss_ensemble = tick_loss(alpha=0.05, returns= ret_list_12m[i], var=Var_list_12m[i])
    tick_loss_list_12.append(tick_loss_ensemble)

tick_loss_list_12m=[]
for i in range(len(tick_loss_list_12)):
    tick_loss_list_12m.append(tick_loss_list_12[i]/1)

tick_loss_list_12m

[7.88107322583407,
 5.7043180632690555,
 5.451784247544036,
 8.975221491853723,
 7.259032355949508,
 7.743395546962186,
 5.2469287407436935,
 8.685567827922373,
 45.35743841781948]